In [1]:
import torch
import torch.nn as nn
import math

# InputEmbeddings

=> input to input embeddings = list of indices of tokens

=> max index = vocab_size

=> output = each index correspond to unique matrix of dimension[dim]

In [2]:
class InputEmbeddings(nn.Module):

    def __init__(self, vocab_size: int, d_model: int ) -> None:
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size,d_model)

    def forward(self,x):
        #seq_len = number of words
        # (Batch, seq_len) --> (Batch,seq_len, d_model)
       return self.embedding(x) * math.sqrt(self.d_model)


## Positional Encoding

### Purpose of Positional encoding?
- add to the embedding output but why ??
- what to add is decided by formula
- different values added to even(sine) and odd(cos) positions in the 
output of input embedding

- output of positional ecoding => seq_len number of [d_model] vectors

### Where to add dropout and where not to??

In [3]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)
        # Create a matrix of shape (seq_len, d_model)
        pe = torch.zeros(seq_len, d_model)
        # Create a vector of shape (seq_len)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) # (seq_len, 1)
        # Create a vector of shape (d_model)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)) # (d_model / 2)
        # Apply sine to even indices
        pe[:, 0::2] = torch.sin(position * div_term) # sin(position * (10000 ** (2i / d_model))
        # Apply cosine to odd indices
        pe[:, 1::2] = torch.cos(position * div_term) # cos(position * (10000 ** (2i / d_model))
        # Add a batch dimension to the positional encoding
        pe = pe.unsqueeze(0) # (1, seq_len, d_model)
        # Register the positional encoding as a buffer
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False) # (batch, seq_len, d_model)
        return self.dropout(x)    

# LayerNormalization
=> tensor that is to be considered a module parameter

=> used for defining learnable parameters

=> tensor will be added to list of the model's parameters

=> model.parameters()

=> model.state.dict() # will appear in parameter list

In [4]:

class LayerNormalization(nn.Module):

    def __init__(self,eps: float = 10**-6 ) -> None:
        super().__init__()

        self.eps = eps

        # Create learable parameter
        self.alpha = nn.Parameter(torch.ones(1))
        self.bias = nn.Parameter(torch.ones(1))


    def forward(self, x):

       mean = x.mean(dim= -1, keepdim= True)     
       std = x.std(dim= -1, keepdim= True)
       return self.alpha * (x - mean)/(std + self.eps) + self.bias


##  dim and keepdim
=> mean calculated along a specified dimension

In [5]:
torch.manual_seed(seed=0)
a = torch.randn(3,3)
## by default takes the last dimension
print(f"simple mean with keepdim = False {a.mean(dim = -1)}")
print(f"Keep dim = True mean {a.mean(dim =-1,keepdim=True)}")
# print(f"simple mean {a.mean()}")

simple mean with keepdim = False tensor([-0.3104, -0.6382,  0.1740])
Keep dim = True mean tensor([[-0.3104],
        [-0.6382],
        [ 0.1740]])


# FeedForward 
=> Fully connected layer

=> Linear layer => ReLU => Linear layer

=> d_model x d_ff

=> d_ff x d_model

In [6]:
class FeedForwardBlock(nn.Module):

    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        super().__init__()

        self.linear1 = nn.Linear(d_model, d_ff) #w1 and b1
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(d_ff, d_model) #w2 and b2


    def forward(self, x):

        # first linear transformation
        # (Batch, seq_len,d_model) --> (Batch, seq_len, d_ff)
        x = self.linear1(x)
        # apply relu
        x  = torch.relu(x)
        # apply dropout to reduce overfitting
        x = self.dropout(x)
        # second linear transformation    
        # (Batch, seq_len, d_ff) --> (Batch, seq_len,d_model)
        x = self.linear2(x)

        return x

# ReLU (Rectified Linear Unit) Activation function
=> introduces nonlinearity

=>ReLU(x)=max(0,x)

=> this function replaces 0 value when x is smaller or equal to zero,otherwise it remains x

In [7]:
test_relu = torch.tensor([-1,2,-3,4])

reluLayer = nn.ReLU()
reluLayer(test_relu)

tensor([0, 2, 0, 4])

# MultiheadAttention Block

In [8]:
class MultiheadAttentionBlock(nn.Module):

    def __init__(self, d_model: int, dropout: int, h: int) -> None:
        super().__init__()

        self.d_model = d_model
        self.dropout = nn.Dropout(dropout)
        self.h = h 

        ## Customizing AssertionError =>
        assert d_model % h == 0, "d_model not completely divisible by h"

        self.d_k = d_model//h # // for integral division

        ## Creating LinearMatrixes 
        self.w_q = nn.Linear(d_model, d_model) #Wq
        self.w_k = nn.Linear(d_model, d_model) #Wk
        self.w_v = nn.Linear(d_model, d_model) #Wv

        self.w_o = nn.Linear(d_model, d_model) #Wo


    ## Static method
    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):

        d_k = query.shape[-1] # (Batch,h, seq_len, d_k)

        # (Batch,h, seq_len, d_k) @ (Batch,h, d_k,seq_len)
        attention_scores = query @ key.transpose(-1,-12) #(Batch,h, seq_len, seq_len)

        ## Applying Mask before Softmax        

        # apply mask
        if mask is not None:
            attention_scores.masked_fill(mask == 0, -1e9)

        attention_scores = attention_scores.softmax(dim = -1) #(Batch,h, seq_len, seq_len)

        if dropout is not None:
            attention_scores = dropout(attention_scores)

        # returning a Tuple
        ## (Batch,h, seq_len, seq_len) 2 (Batch,h, seq_len, d_k) --> (Batch,h, seq_len, d_k)
        return (attention_scores @ value), attention_scores    




    def forward(self, q,k,v, mask):

        query = self.w_q(q)    # (Batch, Seq_len, d_model) -->(Batch, Seq_len, d_model)
        key = self.w_k(k)    # (Batch, Seq_len, d_model) -->(Batch, Seq_len, d_model)
        value = self.w_v(v)    # (Batch, Seq_len, d_model) -->(Batch, Seq_len, d_model)

        ## Dividing the query, key and value matrixes into head number of matrices

        #(Batch, seq_len, d_model) --> (Batch, seq_len, h, d_k) --> (Batch, h, seq_len, d_k)
        ### Converting it directly is Incorrect. That much has been validated
        ### Why it is incorrect ? Yet to research

        query = query.view(query.shape[0], query.shape[1],self.h, self.d_k).transpose(1,2)
        key = key.view(key.shape[0], key.shape[1],self.h, self.d_k).transpose(1,2)
        value = value.view(value.shape[0], value.shape[1],self.h, self.d_k).transpose(1,2)


        ## Applying attention function
        x,self.attention_scores = MultiheadAttentionBlock().attention(query, key, value,mask,self.dropout)

        # (Batch,h, seq_len, d_k) --> (Batch, seq_len , h, d_k ) --> (Batch, seq_len, h * d_k)
        x = x.transpose(1,2).contiguous().view(x.shape[0], -1, self.h * self.d_k)

        # (Batch, seq_len, d_model) --> (Batch, seq_len, d_model)
        return self.w_o(x) 

        

        


## Residual Connections

=> Manages skip connection

In [9]:
class ResidualConnections(nn.Module):

    def __init__(self,dropout: float ) -> None:
        super().__init__()
        self.dropout = nn.Dropout(droput)
        self.norm = LayerNormalization()


    def forward(self, x, sublayer):

        ### we are here applying normalization first 
        # then the sublayer
        return x+ self.dropout(sublayer(self.norm(x)))    


# Encoder Block

=> Encoder block made up of two layers

=> One residual made up of multihead and add and norm

=> Second residual made up of feed forward and add and norm

In [10]:
class EncoderBlock(nn.Module):

    def __init__(self, self_attention_block: MultiheadAttentionBlock, feed_forward_block: FeedForwardBlock,dropout:float) -> None:
        super().__init__()

        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnections(droput=dropout) for _ in range(2)])


    def forward(self, x, mask):
        # first residual connection
        ## why Lambda functtion ??
        x = self.residual_connections[0](x,lambda x: self.self_attention_block(x,x,x,mask))

        # Second residual connection
        ## Why doesn't this take x ??
        x = self.residual_connections[1](x, self.feed_forward_block(x))
        return x
            

        

# Encoder

In [11]:
class Encoder(nn.Module):

    def __init__(self, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization()

    def forward(self,x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return x            

# DecoderBlock
=> Decoder consists of three layers each followed by add and Norm

=> Self attentionnorm

=> Cross attention(q from decoder and k and v from encoder output)

=> Feed Forward 

In [12]:
class DecoderBlock(nn.Module):

    def __init__(self,dropout:float, self_attention_block: MultiheadAttentionBlock, 
                 cross_attention_block: MultiheadAttentionBlock, feed_forward_block: FeedForwardBlock ) -> None:
        super().__init__()

        self.self_attention_block = self_attention_block
        self.cross_attention_block = cross_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnections(dropout= dropout) for _ in range(3)])

    def forward(self, x, encoder_output,src_mask, tgt_mask):

        # first residual connection
        x = self.residual_connections[0](x,lambda x: self.self_attention_block(x,x,x,tgt_mask))

        # second residual connection
        x = self.residual_connections[1](x,lambda x: self.cross_attention_block(x,encoder_output,encoder_output,src_mask))
     
        # third residual connection
        x = self.residual_connections[2](x, self.feed_forward_block(x))

        return x

# Decoder

In [13]:
class Decoder(nn.Module):

    def __init__(self, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization()


    def forward(self, x, encoder_output, src_mask, tgt_mask):
        for layer in self.layers:
            layer(x, encoder_output,src_mask, tgt_mask)

        return self.norm(x)        

# ProjectionLayer
=> The linear layer on the output of the decoder

=> Converts (Batch, seq_len, d_model) --> (Batch, seq_len, vocab_size)

=> Apply log_softmax at the end

In [14]:
class ProjectionLayer(nn.Module):

    def __init__(self, d_model: int, vocab_size: int) -> None:
        super().__init__()
        
        self.projection = nn.Linear(d_model,vocab_size)


    def forward(self,x):
        return torch.log_softmax(self.projection(x), dim=-1)    



# Transformer

=> encoder + input embeddings + positional embeddings

=> decoder + input embeddings + positional encodings

=> Projection Layer

In [15]:
class Transformer(nn.Module):

    def __init__(self, encoder: Encoder, decoder: Decoder, src_embed: InputEmbeddings, tgt_embed: InputEmbeddings, src_pos: PositionalEncoding, tgt_pos: PositionalEncoding, projection_layer: ProjectionLayer) -> None:
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.src_pos = src_pos
        self.tgt_pos = tgt_pos
        self.projection_layer = projection_layer


    #### Instead of having a single forward
    # we will have three functions
    # encoder, decoder and project

    def encode(self,src, mask):
        src = self.src_embed(src) # input embeddings
        src = self.src_pos(src) # positional embeddings
        return self.encoder(src, mask) # then to the encoder
    
    def decode(self,x, encoder_output, src_mask, tgt_mask):
        x = self.tgt_embed(x)
        x =self.tgt_pos(x)
        return self.decoder(x, encoder_output, src_mask, tgt_mask)
    

    def project(self,x):
        return self.projection_layer(x)

